# Cheat Detector
Similarity-based cheat detector for written exams with multiple questions.

In [ ]:
from detector_de_cola import DetectorDeCola
import numpy as np

from IPython.display import clear_output, display
import ipywidgets
import zipfile
import io

In [ ]:
detector_de_cola = DetectorDeCola()

In [ ]:
file_upload_button = ipywidgets.FileUpload()

In [ ]:
class ThresholdSlider(object):

    def __init__(self):
        self.widget = ipywidgets.FloatSlider(
            value=0.5,  # default value
            min=0, max=1, step=0.05
        )
        self.output = ipywidgets.Output()
        self.has_been_instantiated = False

    def display(self):
        if self.has_been_instantiated:
            # with threshold_slider.output:
            #     clear_output(True)
            #     display(threshold_slider.widget)
            with self.output:
                clear_output(True)
                display(self.widget)
        else:
            # display(threshold_slider.output)
            display(self.output)
            self.has_been_instantiated = True

threshold_slider = ThresholdSlider()

In [ ]:
class WhichQuestionsWidget(object):

    def __init__(self):
        self.widget = self.create_widget(1, None, None)
        self.output = ipywidgets.Output()
        self.has_been_instantiated = False

    def observe(self, function, names):
        self.function_to_observe = function
        self.names_to_observe = names

    def create_widget(self, question_quantity, function_to_observe, names_to_observe):
        which_questions = [i+1 for i in range(question_quantity)]
        widget = ipywidgets.SelectMultiple(
            options=which_questions,
            value=which_questions,  # default value
            disabled=False
        )
        widget.observe(function_to_observe, names_to_observe)
        return widget

    def display(self, question_quantity):
        if self.has_been_instantiated:
            self.widget = self.create_widget(
                question_quantity, self.function_to_observe, self.names_to_observe
            )
            with self.output:
                clear_output(True)
                display(self.widget)
        else:
            display(self.output)
            self.has_been_instantiated = True


which_questions_widget = WhichQuestionsWidget()

In [ ]:
closest_students_plot = ipywidgets.Output()

## How to use
For the correct functioning of the cheat detector, it is necessary to send a zip file that contains each student exam. Each exam should be a text file (.txt) in the following format. Don't forget the blank line between each question answer.

```
Answer to question 1

Answer to question 2

Answer to question 3
```

Here's an [unzipped folder](https://github.com/projeto-de-algoritmos/Grafo1_Joao_Lucas_Leonardo_Miranda/blob/main/mock_data/prova_1) example.

In [ ]:
display(file_upload_button)

### Similarity Threshold

In [ ]:
threshold_slider.display()

### Which questions should be considered?

In [ ]:
which_questions_widget.display(question_quantity=1)

### Closest Answers Graph


In [ ]:
display(closest_students_plot)

In [ ]:
def plot_graph(unuseful_arg):
    global output
    threshold = threshold_slider.widget.value
    selected_questions = np.asarray(which_questions_widget.widget.value)

    if selected_questions.shape[0] <= 0:
        return

    mean_sim_matrix = np.mean(
        np.take(
            sim_matrix_list,
            indices=selected_questions-1,
            axis=0
        ),
        axis=0
    )

    closest_answers_graph = detector_de_cola.sim_matrix_to_closest_answers(
        mean_sim_matrix, threshold
    )

    with closest_students_plot:
        clear_output(True)
        detector_de_cola.plot_closest_answers(
            closest_answers_graph
        )

In [ ]:
def on_zip_upload(unuseful_arg):
    global sim_matrix_list, name_list, exam_list, question_answers_list, zip_file
    zip_bytes = file_upload_button.data[-1]
    zip_file = zipfile.ZipFile(io.BytesIO(zip_bytes), mode="r",)

    filename_list = zip_file.namelist()
    exam_list = np.asarray([[i.decode() for i in zip_file.read(filename).splitlines() if i != b'']
                            for filename in filename_list if '.txt' ==  filename[-4:]])
    zip_file.close()

    name_list = [filename.replace('.txt', '') for filename in filename_list]
    question_answers_list = exam_list.T

    sim_matrix_list = np.asarray(
        [detector_de_cola.predict_sim_matrix(question_answers)
        for question_answers in question_answers_list]
    )

    # plot graph for the first time
    plot_graph(unuseful_arg=None)

    threshold_slider.display()
    which_questions_widget.display(
        question_quantity=len(sim_matrix_list)
    )

In [ ]:
file_upload_button.observe(on_zip_upload, names=['data'])
threshold_slider.widget.observe(plot_graph, names='value')
which_questions_widget.observe(plot_graph, names='value')